# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

from config import gkey

gmaps.configure(api_key=gkey)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/Weather_data.csv"

# Load the CSV file into a Pandas DataFrame
city_data_dfs = pd.read_csv(file_path)
# Display sample data
city_data_dfs.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,62.64,70,100,14.67,overcast clouds
1,1,Roald,NO,62.5833,6.1333,57.33,87,100,11.50,overcast clouds
2,2,Nanortalik,GL,60.1432,-45.2371,36.37,74,5,4.21,clear sky
3,3,Sisimiut,GL,66.9395,-53.6735,31.60,84,100,6.55,overcast clouds
4,4,Antofagasta,CL,-23.6500,-70.4000,58.28,75,25,4.14,scattered clouds
5,5,Batemans Bay,AU,-35.7167,150.1833,65.10,56,81,9.48,broken clouds
6,6,Busselton,AU,-33.6500,115.3333,59.76,63,0,13.33,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,76.39,78,28,19.46,scattered clouds
8,8,Mahebourg,MU,-20.4081,57.7000,70.25,78,20,6.91,few clouds
9,9,Ketchikan,US,55.3422,-131.6461,52.88,78,99,4.29,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temparature you would like for your trip?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temparature you would like for your trip?"))

What is the minimum temparature you would like for your trip?60
What is the maximum temparature you would like for your trip?90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_dfs.loc[(city_data_dfs["Max Temp"] <= max_temp) & \
                                        (city_data_dfs["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,62.64,70,100,14.67,overcast clouds
5,5,Batemans Bay,AU,-35.7167,150.1833,65.10,56,81,9.48,broken clouds
7,7,Atuona,PF,-9.8000,-139.0333,76.39,78,28,19.46,scattered clouds
8,8,Mahebourg,MU,-20.4081,57.7000,70.25,78,20,6.91,few clouds
12,12,Hambantota,LK,6.1241,81.1185,77.34,94,20,3.44,few clouds
13,13,Rikitea,PF,-23.1203,-134.9692,71.42,65,89,19.66,overcast clouds
15,15,Rajauri,IN,33.3833,74.3000,67.93,44,0,6.13,clear sky
16,16,Avarua,CK,-21.2078,-159.7750,78.85,73,40,20.71,scattered clouds
17,17,Falconara Marittima,IT,43.6233,13.4031,62.10,64,2,4.61,clear sky
21,21,Souillac,MU,-20.5167,57.5167,70.18,78,20,6.91,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,62.64,70,100,14.67,overcast clouds
5,5,Batemans Bay,AU,-35.7167,150.1833,65.10,56,81,9.48,broken clouds
7,7,Atuona,PF,-9.8000,-139.0333,76.39,78,28,19.46,scattered clouds
8,8,Mahebourg,MU,-20.4081,57.7000,70.25,78,20,6.91,few clouds
12,12,Hambantota,LK,6.1241,81.1185,77.34,94,20,3.44,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
0,East London,ZA,62.64,overcast clouds,-33.0153,27.9116
5,Batemans Bay,AU,65.10,broken clouds,-35.7167,150.1833
7,Atuona,PF,76.39,scattered clouds,-9.8000,-139.0333
8,Mahebourg,MU,70.25,few clouds,-20.4081,57.7000
12,Hambantota,LK,77.34,few clouds,6.1241,81.1185
13,Rikitea,PF,71.42,overcast clouds,-23.1203,-134.9692
15,Rajauri,IN,67.93,clear sky,33.3833,74.3000
16,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750
17,Falconara Marittima,IT,62.10,clear sky,43.6233,13.4031
21,Souillac,MU,70.18,few clouds,-20.5167,57.5167


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,62.64,overcast clouds,-33.0153,27.9116,
5,Batemans Bay,AU,65.10,broken clouds,-35.7167,150.1833,
7,Atuona,PF,76.39,scattered clouds,-9.8000,-139.0333,
8,Mahebourg,MU,70.25,few clouds,-20.4081,57.7000,
12,Hambantota,LK,77.34,few clouds,6.1241,81.1185,
13,Rikitea,PF,71.42,overcast clouds,-23.1203,-134.9692,
15,Rajauri,IN,67.93,clear sky,33.3833,74.3000,
16,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
17,Falconara Marittima,IT,62.10,clear sky,43.6233,13.4031,
21,Souillac,MU,70.18,few clouds,-20.5167,57.5167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,62.64,overcast clouds,-33.0153,27.9116,Tu Casa
5,Batemans Bay,AU,65.10,broken clouds,-35.7167,150.1833,Batemans Bay Marina Resort
7,Atuona,PF,76.39,scattered clouds,-9.8000,-139.0333,Villa Enata
8,Mahebourg,MU,70.25,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Hambantota,LK,77.34,few clouds,6.1241,81.1185,Bungalow 63
13,Rikitea,PF,71.42,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Rajauri,IN,67.93,clear sky,33.3833,74.3000,Hotel Ashoka
16,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
17,Falconara Marittima,IT,62.10,clear sky,43.6233,13.4031,Touring Hotel
21,Souillac,MU,70.18,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [12]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,62.64,overcast clouds,-33.0153,27.9116,Tu Casa
5,Batemans Bay,AU,65.10,broken clouds,-35.7167,150.1833,Batemans Bay Marina Resort
7,Atuona,PF,76.39,scattered clouds,-9.8000,-139.0333,Villa Enata
8,Mahebourg,MU,70.25,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Hambantota,LK,77.34,few clouds,6.1241,81.1185,Bungalow 63
13,Rikitea,PF,71.42,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Rajauri,IN,67.93,clear sky,33.3833,74.3000,Hotel Ashoka
16,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
17,Falconara Marittima,IT,62.10,clear sky,43.6233,13.4031,Touring Hotel
21,Souillac,MU,70.18,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [13]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [14]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Tu Casa</dd>\n<dt>City</dt><dd>East London</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 62.64 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Batemans Bay Marina Resort</dd>\n<dt>City</dt><dd>Batemans Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>broken clouds and 65.1 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 76.39 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shandrani Beachcomber Resort & Spa</dd>\n<dt>City</dt><dd>Mahebourg</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>few clouds and 70.25 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bungalow 63</dd>\n<dt>City</dt><dd>Hambantota</dd>\n<dt>Country</dt><dd>LK</dd>\n<dt>Current Weather</dt><dd>few clouds and 77.34 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People Thank

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
# Display sample data
locations.head(10)

,Lat,Lng
0,-33.0153,27.9116
5,-35.7167,150.1833
7,-9.8000,-139.0333
8,-20.4081,57.7000
12,6.1241,81.1185
13,-23.1203,-134.9692
15,33.3833,74.3000
16,-21.2078,-159.7750
17,43.6233,13.4031
21,-20.5167,57.5167


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [18]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))